### Домашнее задание <a class="anchor" id="hw"></a><center>

In [422]:
import numpy as np
import matplotlib.pyplot as plt

In [423]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [424]:
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 5.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [425]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [426]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [427]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [428]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

### Задание 1

#### *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [429]:
def calc_logloss(y, y_pred):
    
#-----------------------------------------------------------------------------------------------
# Было
#     err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
#-----------------------------------------------------------------------------------------------
# Стало
# В массиве y_pred заменяю "1" на число, близкое к "1" (1 - 1e-12)

    y_pred_new = y_pred.copy()
    y_pred_new[y_pred_new == 1] = 1 - 1e-12
    err = - np.mean(y * np.log(y_pred_new) + (1.0 - y) * np.log(1.0 - y_pred_new))
#-----------------------------------------------------------------------------------------------
    return err

In [430]:
# Проверка - Плохой пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782941313

### Задание 2

#### Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [431]:
def eval_model(X, y, iterations, eta=1e-4):
    result = []
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
#------------------------------------------------------------------------------------------------
# Стало
    err_prev = np.inf
    err_delta = np.inf
    err_precision = 1e-7
    k = iterations // 5      # кратность вывода лога
#------------------------------------------------------------------------------------------------
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
#------------------------------------------------------------------------------------------------        
# Было
#         if i % (iterations / 10) == 0:
#     return W
#------------------------------------------------------------------------------------------------
# Стало
        if (i % k == 0) or (i == iterations - 1):
            print(f'i = {i}; W = {W}; err = {err}; err_prev = {err_prev}')

        # выход из цикла, если ошибка достигла заданной точности
        err_delta = abs(err - err_prev)
        
        if (err_delta < err_precision):
            print(f'\tBreak by error precision:\n\t'\
            f'err_prev = {round(err_prev, 8):8}; err = {round(err, 8):8}; err_delta = {round(err_delta, 8):8}')
            break
            
        err_prev = err
    result = [i + 1, W, err]
    return result
#------------------------------------------------------------------------------------------------


In [432]:
# перебор нескольких вариантов для разного числа итераций, определение наименьшей ошибки

iter_list = [10**i for i in range(1, 7)]
eta = 1e-4
res_list = []     # список полученных результатов 
var_best = -1     # номер лучшего варианта
err_min = np.inf  # наименьшая ошибка
i = 0

print(f'list iteration = {iter_list}')
print(f'eta = {eta}')
print (100 * '-')

for i, iter in enumerate(iter_list):

    print(f'Variant: {i}; Iter = {iter}')
    print (100 * '-')
    
    res = eval_model(X_st, y, iter, eta=1e-4)
    res_list.append(res)
    
    # ошибка для i-го варианта
    err = res[2]
    if (err < err_min):
        err_min = err
        var_best = i

    print (100 * '-')
    print (f'\tResult: Iter = {res[0]}; W = {res[1]}; err = {round(res[2], 8)}')

    print ('\n' + 100 * '-')
    
# список ошибок для всех вариантов
err_list = []
for res in res_list:
    err_list.append(round(res[2], 8))

print(f'List errors:\n\t {err_list}')

# вывод лучшего результата
W = res_list[var_best][1]
print (f'\nBest result:\n\tVariant = {var_best}; Iter = {res_list[var_best][0]}; \
W = {res_list[var_best][1]}; err = {round(res_list[var_best][2], 8)}')


list iteration = [10, 100, 1000, 10000, 100000, 1000000]
eta = 0.0001
----------------------------------------------------------------------------------------------------
Variant: 0; Iter = 10
----------------------------------------------------------------------------------------------------
i = 0; W = [ 0.49667621 -0.13840939  0.6476858   1.52297324]; err = 1.1785958344356262; err_prev = inf
i = 2; W = [ 0.49660035 -0.13869953  0.64768032  1.52286003]; err = 1.178081850227778; err_prev = 1.1783388206117205
i = 4; W = [ 0.4965245  -0.13898962  0.64767483  1.52274683]; err = 1.1775680398244004; err_prev = 1.1778249232949523
i = 6; W = [ 0.49644866 -0.13927966  0.64766933  1.52263365]; err = 1.17705440331467; err_prev = 1.1773111998272563
i = 8; W = [ 0.49637284 -0.13956965  0.64766382  1.52252049]; err = 1.1765409407877048; err_prev = 1.1767976502977708
i = 9; W = [ 0.49633493 -0.13971462  0.64766106  1.52246391]; err = 1.1762842747955884; err_prev = 1.1765409407877048
----------------

### Задание 3.

#### Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [433]:
print(f'W = {W}')
print(f'X = {X_st}')

W = [-1.51040014 -0.92057939  0.76778342  2.49704822]
X = [[ 1.          1.         -0.97958969  1.        ]
 [ 1.          1.         -0.56713087  1.        ]
 [ 1.          2.         -0.46401617  2.        ]
 [ 1.          5.         -0.77336028  1.        ]
 [ 1.          3.          0.97958969  2.        ]
 [ 1.          0.         -0.36090146  1.        ]
 [ 1.          5.          1.08270439  3.        ]
 [ 1.         10.          2.11385144  3.        ]
 [ 1.          1.         -1.08270439  1.        ]
 [ 1.          2.          0.05155735  2.        ]]


In [434]:
def sigmoid(z):
    res = 1.0 / (1 + np.exp(-z))
    return res

In [435]:
def calc_pred_proba(w, X):
    return np.squeeze(sigmoid(np.dot(X, w)))

In [436]:
P_list = calc_pred_proba(W, X_st)

print('Вероятность принадлежности объектов классу 1:')
for i, p in enumerate(P_list):
    print(f'\ti = {i};\t P = {round(p, 4):4}')

Вероятность принадлежности объектов классу 1:
	i = 0;	 P = 0.3349
	i = 1;	 P = 0.4087
	i = 2;	 P = 0.7835
	i = 3;	 P = 0.0146
	i = 4;	 P = 0.8137
	i = 5;	 P = 0.6703
	i = 6;	 P = 0.9011
	i = 7;	 P = 0.1677
	i = 8;	 P = 0.3175
	i = 9;	 P = 0.8432


### Задание 4

#### Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [437]:
def calc_pred(w, X):
    
    m = X.shape[0]
    y_predicted = np.zeros(m)
    P = calc_pred_proba(w, X)

    # Порог отнесения к классам - P = 0.5
    for i in range(P.shape[0]):
        if (P[i] > 0.5): 
            y_predicted[i] = 1
        else:
            y_predicted[i] = 0

    return list(map(int, y_predicted))

In [438]:
y_pred = calc_pred(W, X_st)
print(y_pred)

print('\nПредсказанная принадлежность объектов к классам:')
for i, class_pred in enumerate(y_pred):
    print(f'\ti = {i};\t class_pred = {class_pred};\t class fact = {int(y[i])};\t error = {abs(class_pred - int(y[i]))}')

[0, 0, 1, 0, 1, 1, 1, 0, 0, 1]

Предсказанная принадлежность объектов к классам:
	i = 0;	 class_pred = 0;	 class fact = 0;	 error = 0
	i = 1;	 class_pred = 0;	 class fact = 0;	 error = 0
	i = 2;	 class_pred = 1;	 class fact = 1;	 error = 0
	i = 3;	 class_pred = 0;	 class fact = 0;	 error = 0
	i = 4;	 class_pred = 1;	 class fact = 1;	 error = 0
	i = 5;	 class_pred = 1;	 class fact = 0;	 error = 1
	i = 6;	 class_pred = 1;	 class fact = 1;	 error = 0
	i = 7;	 class_pred = 0;	 class fact = 0;	 error = 0
	i = 8;	 class_pred = 0;	 class fact = 1;	 error = 1
	i = 9;	 class_pred = 1;	 class fact = 1;	 error = 0


### Задание 5

#### *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

#### Accuracy

In [439]:
# Оценка качества: показатель Accuracy

def calc_accuracy(y_pred, y_test):
    return np.mean(y_pred == y_test)

In [440]:
accuracy = calc_accuracy(y_pred, y)
print(f'Accuracy = {accuracy}')

Accuracy = 0.8


#### Error Matrix

In [441]:
# Оценка качества: Матрица ошибок

def calc_error_matrix(y_pred, y_test):

    TP = FP = FN = TN = 0
    for i, pred in enumerate(y_pred):
    
        test = y_test[i]
        if pred == 1 and test == 1:
            TP += 1                    # True Positive
        elif pred == 1 and test == 0:
            FP += 1                    # False Positive
        elif pred == 0 and test == 1:
            FN += 1                    # False Negative
        else:
            TN += 1                    # True Negative
    
    # сама матрица ошибок
    error_matrix = np.zeros( (2, 2), dtype = np.int32)
    
    error_matrix[0, 0] = TP
    error_matrix[0, 1] = FP
    error_matrix[1, 0] = FN
    error_matrix[1, 1] = TN
    
    return error_matrix

In [442]:
a = np.zeros( (2, 2), dtype = np.int32)

In [443]:
print('Матрица ошибок:\n')
print(f'{calc_error_matrix(y_pred, y)}')

Матрица ошибок:

[[4 1]
 [1 4]]


#### Precision

In [444]:
# Оценка качества: Точность (Precision)

def calc_precision(y_pred, y_test):

    error_matrix = calc_error_matrix(y_pred, y_test)
    result = 0
    
    if error_matrix[0, 0] + error_matrix[0, 1] != 0:
        result = error_matrix[0, 0] / (error_matrix[0, 0] + error_matrix[0, 1])
    
    return result

In [445]:
print(f'Точность = {calc_precision(y_pred, y)}')

Точность = 0.8


#### Recall

In [446]:
# Оценка качества: Полнота (Recall)

def calc_recall(y_pred, y_test):

    error_matrix = calc_error_matrix(y_pred, y_test)
    result = 0
    
    if error_matrix[0, 0] + error_matrix[1, 0] != 0:
        result = error_matrix[0, 0] / (error_matrix[0, 0] + error_matrix[1, 0])
    
    return result

In [447]:
print(f'Полнота = {calc_recall(y_pred, y)}')

Полнота = 0.8


#### F1 score

In [448]:
# Оценка качества: F1 score

def calc_F1_score(y_pred, y_test):

    precision = calc_precision(y_pred, y_test)
    recall    = calc_recall(y_pred, y_test)
    result = 0
    
    if precision + recall != 0:
        result = round(2 * precision * recall / (precision + recall), 6)
    
    return result

In [449]:
print(f'F1 score = {calc_F1_score(y_pred, y)}')

F1 score = 0.8


### Задание 6

#### Могла ли модель переобучиться? Почему?

In [450]:
print(f'Веса W = {W}')

Веса W = [-1.51040014 -0.92057939  0.76778342  2.49704822]


Веса у модели получились небольшие, поэтому, скорее всего, переобучения модели не было.